## Data splitting and some auxiliary functions

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"    #to prevent the kernel from stopping
import pandas as pd
import numpy as np
path = "/Users/jerryzhang/Downloads/merged_dataset.csv"
df = pd.read_csv(path)

df = df[df['DATE'].between(19570101, 20201231, inclusive='both')] #also include data from 2016-2020
data_top = df.head() 
print(data_top)
covardata = list(df.columns.values)
print(covardata) #to demonstrate z_{i,t} variables' names
df.describe()    #some of the summary statistics

## Splitting function

In [ ]:
# I learn this user-friendly splitting method thanks to Cao & Chu from the 2022 cohort of this class

# We first define a function for splitting the dataset into different categories, such as testing and training, etc.
# We want a simple form like No. of years and the starting year
# The original date is in YYMMDD form, but not easy to deal with
# note that this splitting scheme is based on the file:merged_dataset.csv
#one has to read the data first

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #to prevent the kernel from stopping
import pandas as pd
import numpy as np


def spltdata(yrbegin, yrlength):                  
    datebegin = yrbegin * 10000 + 101                                                   #convert to YYYY0101
    dateend  = (yrbegin+yrlength-1) * 10000 + 1231                                      #convert to YYYY1231
    
    dataset  = df[df['DATE'].between(datebegin, dateend, inclusive='both')].values      #truncate
    
#create industry dummies

    indx_sic2 = covardata.index("sic2")
    indx_mve0 = covardata.index("mve0")
    indx_bm = covardata.index("b/m")
    
#remove some indexes

    cht_index = list(range(6, indx_bm))
    
    cht_index.remove(indx_sic2)
    cht_index.remove(indx_mve0)
    
    cht_index.insert(0, 2)

    
#when running different models, the data possess a tensor structure
#the input data has P=920 dimension, according to the paper

    dataset_size = dataset.shape[0]
    sic2 = np.zeros([dataset_size, 1])
    output_data = np.zeros([dataset_size, 1])
    pred_data = np.ones([dataset_size, 9])
    input_data = np.zeros([dataset_size, 920])    #all are tensor
    
    sic2_list = [1, 2, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25,
                26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44,
                45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62,
                63, 64, 65, 67, 70, 72, 73, 75, 76, 78, 79, 80, 81, 82, 83, 84, 86, 87,
                89, 99]

#better demonstration in each loop
    first_flag = True
    year = yrbegin
    tidx_list = []
    for i in range(yrlength):
        month = 1
        for j in range(12): 
            datebegin = int(year*1e4 + month*1e2 + 0)
            dateend = int(year*1e4 + month*1e2 + 32)
            df_process = df[df['DATE'].between(datebegin, dateend, inclusive='both')]
            df_process_np = df_process.values
            
            for col in cht_index:
                single_col = df_process_np[:,col]
                if np.isnan(single_col).all():
                    mid = 0
                    df_process_np[:,col] = mid
                elif np.isnan(single_col).any():
                    mid = np.median(single_col[~np.isnan(single_col)])
                    df_process_np[np.isnan(single_col),col] = mid
                else:
                    continue
            
            if first_flag == True:
                cht_data = df_process_np
                tidx_list.append(df_process_np.shape[0])
                first_flag = False
            else:
                cht_data = np.vstack([cht_data, df_process_np])
                tidx_list.append(df_process_np.shape[0]+tidx_list[-1])

            month = month+1
        year = year+1
    first_flag = True

    sic2 = dataset[:,indx_sic2]  
    output_data = dataset[:,3]  
    pred_data[:,1:] = dataset[:, indx_bm:]  
    cht_data = cht_data[:, cht_index]  
    

    for i in range(dataset_size):
        input_data[i, 0:846] = (pred_data[i, :].reshape(-1,1) * cht_data[i, :]).reshape(1,-1)
        if np.isnan(sic2[i]):
            continue
        else:
            sic_cat = int(sic2[i])
            input_data[i, 846+sic2_list.index(sic_cat)] = 1

    print(np.isnan(output_data).any())

    covardata_cht = []
    for i in range(len(cht_index)):
        covardata_cht.append(covardata[cht_index[i]])

    return input_data, output_data, tidx_list, covardata_cht

##R^2_{oos} and top/bottom 1000

In [ ]:
#  R^2_{oos}
def R2_oos(ygiven, yest):
    
    r2 = 1 - np.square(np.linalg.norm((ygiven - yest)) / np.linalg.norm(y_true)) #vector l_2-norms

    return r2


#  The upper and lower tails values of R^2: top 1000 & bottom 1000 in market
#  Defined based on R2_oos

def segment_data(ygiven, yest, mv, t_interval):

# Combine data and sort by market value
    combined_data = np.hstack([mv.reshape(-1, 1), ygiven.reshape(-1, 1), yest.reshape(-1, 1)])
    sorted_data = combined_data[np.argsort(combined_data[:, 0])]

# Initialize top and bottom segments
    top_segment = np.empty((0, 3))
    bottom_segment = np.empty((0, 3))

# Iterate over time intervals
    for i, interval in enumerate(t_interval):
        start = 0 if i == 0 else t_interval[i - 1]
        end = interval

# Extract data for the current interval
        current_data = sorted_data[start:end, :]

# Update top and bottom segments
        if i == 0:
            top_segment = current_data[-1000:, :]
            bottom_segment = current_data[:1000, :]
        else:
            top_segment = np.vstack([top_segment, current_data[-1000:, :]])
            bottom_segment = np.vstack([bottom_segment, current_data[:1000, :]])

# Return top and bottom segments as a dictionary
    return {"top": top_segment, "bottom": bottom_segment}



def R2_full(ygiven, yest, t_interval, mv):

# Calculate R^2 for the full dataset
    r2_full = R2_oos(ygiven, yest)

# Segment data based on market value and time intervals
    segments = segment_data(ygiven, yest, mv, t_interval)

# Calculate R^2 for top and bottom performing segments
    r2_top = r2_score(segments["top"][:, 1], segments["top"][:, 2])
    r2_bottom = r2_score(segments["bottom"][:, 1], segments["bottom"][:, 2])

# Return R^2 metrics as a tuple
    return r2_full, r2_top, r2_bottom

